# Multiple linear regression

## Import the relevant libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

## Load the data

When we fit the model, we will have to convert all data frames into ndarrays.

In [2]:
# Load the data from a .csv in the same folder
data = pd.read_csv('Multiple linear regression.csv')

# Let's explore the top 5 rows of the dataframe
data.head()

# sat, Rand 1,2,3 are features or input or independent variable
# GPA is the dependent variable or target

,SAT,"Rand 1,2,3",GPA
0,1714,1,2.40
1,1664,3,2.52
2,1760,3,2.54
3,1685,3,2.74
4,1693,2,2.83


In [3]:
# This method gives us very nice descriptive statistics. We don't need this for now, but will later on!
data.describe()

,SAT,"Rand 1,2,3",GPA
count,84.000000,84.000000,84.000000
mean,1845.273810,2.059524,3.330238
std,104.530661,0.855192,0.271617
min,1634.000000,1.000000,2.400000
25%,1772.000000,1.000000,3.190000
50%,1846.000000,2.000000,3.380000
75%,1934.000000,3.000000,3.502500
max,2050.000000,3.000000,3.810000


## Create the multiple linear regression

### Declare the dependent and independent variables

In [4]:
# There are two independent variables: 'SAT' and 'Rand 1,2,3'
x = data[['SAT','Rand 1,2,3']]

# and a single depended variable: 'GPA'
y = data['GPA']

x.shape

(84, 2)

### Regression itself

In [5]:
# creating a linear regression object
# reg is an instance of LinearRegression class
reg = LinearRegression()

# The whole learning process boils down to fitting the regression
# we must shape x variable in 2d array (ex: x.values.reshape(-1,1))
reg.fit(x,y)

LinearRegression()

In [6]:
# Getting the coefficients of the regression
reg.coef_
# Note that the output is an array
# Coefficient are ordered in the way we fed them

array([ 0.00165354, -0.00826982])

In [7]:
# Getting the intercept of the regression
reg.intercept_
# Note that the result is a float as we usually expect a single value

0.29603261264909486

### Calculating the R-squared

In [8]:
# Get the R-squared of the regression
reg.score(x,y)

0.40668119528142843

### Formula for Adjusted R^2

$R^2_{adj.} = 1 - (1-R^2)*\frac{n-1}{n-p-1}$

The R-squared is a universal measure to evaluate how well linear regressions fare and compare.

reg.score(x,y) returns the R-squared of a linear regression.

In [9]:
# Get the shape of x, to facilitate the creation of the Adjusted R^2 metric
x.shape

(84, 2)

In [10]:
# If we want to find the Adjusted R-squared we can do so by knowing the r2, the # observations, the # features
r2 = reg.score(x,y)
# Number of observations is the shape along axis 0
n = x.shape[0]
# Number of features (predictors, p) is the shape along axis 1
p = x.shape[1]

adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
adjusted_r2

0.39203134825134023

### Adjusted R^2 function

In [11]:
# There are different ways to solve this problem
# To make it as easy and interpretable as possible, we have preserved the original code
def adj_r2(x,y):
    r2 = reg.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [12]:
# Here's the result
adj_r2(x,y)

0.39203134825134023

### Compare the R-squared and the Adjusted R-squared

It seems the the R-squared is only slightly larger than the Adjusted R-squared, implying that we were not penalized a lot for the inclusion of 2 independent variables. 

### Compare the Adjusted R-squared with the R-squared of the simple linear regression

Comparing the Adjusted R-squared with the R-squared of the simple linear regression (when only 'size' was used - a couple of lectures ago), we realize that 'Year' is not bringing too much value to the result.

## Feature selection
Full documentation: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html
---Feature selection simplifies models, improves speed and prevents series of unwanted issues arising from having too many features.

In [13]:
# Import the feature selection module from sklearn
# This module allows us to select the most appopriate features for our regression
# There exist many different approaches to feature selection, however, we will use one of the simplest

from sklearn.feature_selection import f_regression

In [14]:
f_regression(x,y)

# There are two output arrays
# The first one contains the F-statistics for each of the regressions
# The second one contains the p-values of these F-statistics

(array([56.04804786,  0.17558437]), array([7.19951844e-11, 6.76291372e-01]))

if a variable has a p-value>0.05, we can disregard it.

In [15]:
# Since we are more interested in the latter (p-values), we can just take the second array
p_values = f_regression(x,y)[1]
p_values

array([7.19951844e-11, 6.76291372e-01])

In [16]:
# To be able to quickly evaluate them, we can round the result to 3 digits after the dot
p_values.round(4)

array([0.    , 0.6763])

## Creating a summary table

In [17]:
# Let's create a new data frame with the names of the features
reg_summary = pd.DataFrame(data = x.columns.values, columns=['Features'])
reg_summary

,Features
0,SAT
1,"Rand 1,2,3"


In [18]:
# Then we create and fill a second column, called 'Coefficients' with the coefficients of the regression
reg_summary ['Coefficients'] = reg.coef_
# Finally, we add the p-values we just calculated
reg_summary ['p-values'] = p_values.round(3)

p-values are one of the best ways to determine if a variable is redundant, but they provide no information whatsoever about How useful a variable is.

In [19]:
# Now we've got a pretty clean summary, which can help us make an informed decision about the inclusion of the variables 
reg_summary

,Features,Coefficients,p-values
0,SAT,0.001654,0.000
1,"Rand 1,2,3",-0.008270,0.676


## Standardization
Full documentation: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [20]:
# Import the preprocessing module
# StandardScaler is one of the easiest and 'cleanest' ways to preprocess your data

from sklearn.preprocessing import StandardScaler

StandardScaler() = a preprocessing module used to standardize (or scale) data

In [21]:
# Create a StandardScaler instance
# scaler will be used to subtract the mean and divide by the standard deviation.

scaler = StandardScaler()

'fit' calculates and stores (scaler object) the mean and standard deviation of each feature.

In [22]:
# Fit the input data (x)
# Essentially we are calculating the mean and standard deviation feature-wise 
# (the mean of 'SAT' and the standard deviation of 'SAT', 
# as well as the mean of 'Rand 1,2,3' and the standard deviation of 'Rand 1,2,3')

scaler.fit(x)

StandardScaler()

transform(x) = transforms the unscaled inputs using the information contained in the scaler object (feature-wise). This is the most common and useful method to tranform new data when you deploy a new model.

In [23]:
# The actual scaling of the data is done through the method 'transform()'
# Let's store it in a new variable, named appropriately

x_scaled = scaler.transform(x)

In [24]:
# The result is an ndarray
x_scaled

array([[-1.26338288, -1.24637147],
       [-1.74458431,  1.10632974],
       [-0.82067757,  1.10632974],
       [-1.54247971,  1.10632974],
       [-1.46548748, -0.07002087],
       [-1.68684014, -1.24637147],
       [-0.78218146, -0.07002087],
       [-0.78218146, -1.24637147],
       [-0.51270866, -0.07002087],
       [ 0.04548499,  1.10632974],
       [-1.06127829,  1.10632974],
       [-0.67631715, -0.07002087],
       [-1.06127829, -1.24637147],
       [-1.28263094,  1.10632974],
       [-0.6955652 , -0.07002087],
       [ 0.25721362, -0.07002087],
       [-0.86879772,  1.10632974],
       [-1.64834403, -0.07002087],
       [-0.03150724,  1.10632974],
       [-0.57045283,  1.10632974],
       [-0.81105355,  1.10632974],
       [-1.18639066,  1.10632974],
       [-1.75420834,  1.10632974],
       [-1.52323165, -1.24637147],
       [ 1.23886453, -1.24637147],
       [-0.18549169, -1.24637147],
       [-0.5608288 , -1.24637147],
       [-0.23361183,  1.10632974],
       [ 1.68156984,

## Regression with scaled features


Before: 'SAT' is ranging between 600 and 2400, while 'Rand 1,2,3' between 1 and 3

In [25]:
# Creating a regression works in the exact same way
reg = LinearRegression()

# We just need to specify that our inputs are the 'scaled inputs'
reg.fit(x_scaled,y)

LinearRegression()

In [26]:
# Let's see the coefficients
reg.coef_

array([ 0.17181389, -0.00703007])

In [27]:
# And the intercept
reg.intercept_

3.330238095238095

## Creating a summary table

intercept = bias, Sample = observation, weight = coefficient

In [28]:
# As usual we can try to arrange the information in a summary table
# Let's create a new data frame with the names of the features
# The ML word for intercept is bias
reg_summary = pd.DataFrame([['Bias'],['SAT'],['Rand 1,2,3']], columns=['Features'])

# Then we create and fill a second column, called 'Weights' with the coefficients of the regression
# Since the standardized coefficients are called 'weights' in ML, this is a much better word choice for our case
# Note that even non-standardized coeff. are called 'weights' 
# 'Weight' is a machine learning word for coefficient.
# but more often than not, when doing ML we perform some sort of scaling
reg_summary['Weights'] = reg.intercept_, reg.coef_[0], reg.coef_[1]

When we perform feature scaling, we dont care if a useless variable is there or not.

In [29]:
# Now we have a pretty clean summary, which can help us make an informed decision about the importance of each feature
# The closer a weight is to 0, the smaller its impact;
# The bigger the weight, the bigger its impact
reg_summary

,Features,Weights
0,Bias,3.330238
1,SAT,0.171814
2,"Rand 1,2,3",-0.007030


## Making predictions with the standardized coefficients (weights)

In [30]:
# For simplicity, let's crete a new dataframe with 2 *new* observations
new_data = pd.DataFrame(data=[[1700,2],[1800,1]],columns=['SAT','Rand 1,2,3'])
new_data

,SAT,"Rand 1,2,3"
0,1700,2
1,1800,1


In [31]:
# We can make a prediction for a whole dataframe (not a single value)
# Note that the output is very strange (different from mine)
reg.predict(new_data)

array([295.39979563, 312.58821497])

In [32]:
# Our model is expecting SCALED features (features of different magnitude)
# In fact we must transform the 'new data' in the same way as we transformed the inputs we train the model on
# Luckily for us, this information is contained in the 'scaler' object
# We simply transform the 'new data' using the relevant method
new_data_scaled = scaler.transform(new_data)

# Let's check the result
new_data_scaled

array([[-1.39811928, -0.07002087],
       [-0.43571643, -1.24637147]])

In [33]:
# Finally we make a prediction using the scaled new data
reg.predict(new_data_scaled)
# The output is much more appropriate, isn't it?

array([3.09051403, 3.26413803])

## What if we removed the 'Random 1,2,3' variable?

In [34]:
# Theory suggests that features with very small weights could be removed and the results should be identical
# Moreover, we proved in 2-3 different ways that 'Rand 1,2,3' is an irrelevant feature
# Let's create a simple linear regression (simple, because there is a single feature) without 'Rand 1,2,3'
reg_simple = LinearRegression()

# Once more, we must reshape the inputs into a matrix, otherwise we will get a compatibility error 
# Note that instead of standardizing again, I'll simply take only the first column of x
x_simple_matrix = x_scaled[:,0].reshape(-1,1)

# Finally, we fit the regression
reg_simple.fit(x_simple_matrix,y)

LinearRegression()

In [35]:
# In a similar manner to the cell before, we can predict only the first column of the scaled 'new data'
# Note that we also reshape it to be exactly the same as x
reg_simple.predict(new_data_scaled[:,0].reshape(-1,1))

array([3.08970998, 3.25527879])

# Train Test Split

## Import the relevant libraries

In [36]:
# In this lesson we will explore the train_test_split module
# Therefore we need no more than the module itself and NumPy
import numpy as np
from sklearn.model_selection import train_test_split

## Generate some data we are going to split


np.arange([start,] stop, [step]) = returns evenly spaced values withing a given interval, By default the output is an ndarray.

In [37]:
# Let's generate a new data frame 'a' which will contain all integers from 1 to 100
# The method np.arange works like the built-in method 'range' with the difference it creates an array
a = np.arange(1,101)

In [38]:
# Let's check it out
a

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])

In [39]:
# Similarly, let's create another ndarray 'b', which will contain integers from 501 to 600
# We have intentionally picked these numbers so we can easily compare the two
# Obviously, the difference between the elements of the two arrays is 500 for any two corresponding elements
b = np.arange(501,601)
b

array([501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513,
       514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526,
       527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539,
       540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552,
       553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565,
       566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578,
       579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591,
       592, 593, 594, 595, 596, 597, 598, 599, 600])

## Split the data
Full documentation: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

train_test_split(x) = splits arrays or matrices into random train and test subsets.

In [40]:
# Let's check out how this works
train_test_split(a)

[array([ 18,  54,  36,  87,  86,  96,  80,  43,  64,  71,  81,  70,  10,
         97,  45,  35,  12,  57,  84,  14,  88,  65,  51,  22,  56,  26,
         89,  15,  82,  76,  62,  94,  58,  38,  42,  44,  24,  60,  55,
        100,  28,  72,   7,  29,  68,  53,  63,  32,  61,  34,   2,   8,
         33,  23,  11,  77,   9,  25,  37,  74,  40,  95,  20,  52,  59,
         49,  99,  19,  83,  30,  17,  93,  21,  47,  41]),
 array([66, 79, 75, 46, 13, 48,  5, 67, 85, 16, 91,  4,  3, 27, 31, 50, 98,
        73, 39, 69, 78,  1, 90,  6, 92])]

In [41]:
a_train, a_test= train_test_split(a)

In [42]:
a_train

array([ 55,  52,  31,  82,  70,  37,  35,  91,  34,  58,  97,  22,  77,
        11,  65,  20,  96,   4,  79,  25,  18,  56,  17,  67,  81,  13,
        27,  33,  89,  64,  43,  46,  59,  66,  19,  16,  40,   5,  98,
         6,  51, 100,  86,  48,  80,   1,  88,  45,  38,   9,  99,  60,
        24,  84,  73,  39,  90,  69,  94,  71,  61,  49,  28,  10,  57,
        47,  85,  41,  87,  53,   2,  74,  36,  72,   3])

In [43]:
a_test

array([26,  7, 30, 44, 29, 92, 12, 54, 23,  8, 95, 15, 42, 14, 50, 75, 68,
       83, 32, 93, 76, 63, 62, 78, 21])

In [44]:
a_train.shape, a_test.shape

((75,), (25,))

You can specify the 'test_size' or the 'train_size' (but the latter is deprecated and will be removed)
 essentially the two have the same meaning 
 Common splits are 75-25, 80-20, 85-15, 90-10

 Finally, you should always employ a 'random_state'
 In this way you ensure that when you are splitting the data you will always get the SAME random shuffle
 
 r-square is likely to change with 1% or 2% just because of the split.

In [45]:
a_train, a_test, b_train, b_test = train_test_split(a, b, test_size=0.2, random_state=365)

## Explore the result

when we split a and b using the train_test_split, their elements are shuffled in the same way.

In [46]:
# aLet's check the shapes
# Basically, we are checking how does the 'test_size' work
a_train.shape, a_test.shape

((80,), (20,))

In [47]:
# Explore manually
a_train

array([ 25,  32,  99,  73,  91,  66,   3,  59,  94,   1,   8,  15,  90,
        54,  31,  20,  77,  82,  30,  35,  95,  42,  38,   7,  11,  50,
        21,  48,   2,  17,  10,  58,  68,  43,  41,  16,  88,  72,  79,
       100,  80,  39,  24,  86,  22,  23,  62,  76,  18,  47,  55,  26,
        60,  19,  71,  64,  51,  63,  65,  28,  12,  78,  13,  44,  75,
        87,  40,   4,  29,  49,  37,  57,  27,  74,   6,  45,  92,  34,
        53,  83])

In [48]:
# Explore manually
a_test

array([ 9, 69, 81, 56, 33, 93, 84, 61, 46, 89, 85, 67, 97,  5, 70, 36, 98,
       96, 14, 52])

In [49]:
b_train.shape, b_test.shape

((80,), (20,))

In [50]:
b_train

array([525, 532, 599, 573, 591, 566, 503, 559, 594, 501, 508, 515, 590,
       554, 531, 520, 577, 582, 530, 535, 595, 542, 538, 507, 511, 550,
       521, 548, 502, 517, 510, 558, 568, 543, 541, 516, 588, 572, 579,
       600, 580, 539, 524, 586, 522, 523, 562, 576, 518, 547, 555, 526,
       560, 519, 571, 564, 551, 563, 565, 528, 512, 578, 513, 544, 575,
       587, 540, 504, 529, 549, 537, 557, 527, 574, 506, 545, 592, 534,
       553, 583])

In [51]:
b_test

array([509, 569, 581, 556, 533, 593, 584, 561, 546, 589, 585, 567, 597,
       505, 570, 536, 598, 596, 514, 552])